In [2]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.13.2 environment at: C:\Users\Hp\OneDrive\Desktop\LLM_OPS\.venv
Resolved 67 packages in 12.17s
Prepared 63 packages in 3m 32s
Installed 63 packages in 15.12s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.15
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.11.0
 + attrs==25.3.0
 + certifi==2025.8.3
 + charset-normalizer==3.4.3
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + flatbuffers==25.9.23
 + frozenlist==1.7.0
 + greenlet==3.2.4
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.1
 + humanfriendly==10.0
 + idna==3.10
 + jiter==0.11.0
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==0.3.27
 + langchain-community==0.3.30
 + langchain-core==0.3.77
 + langchain-text-splitters==0.3.11
 + langsmith==0.4.31
 + marshmallow==3.26.1
 + mpmath==1.3.0
 + multidict==6.6.4
 + mypy-extensions==1.1.0
 + numpy==2.2.6
 + onnxruntime==1.23.0
 + openai==2.0.1
 + opencv-python==4.12.0.88
 + orjson==3.11.3
 + pillow==11.3.0
 + propcache

In [4]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [5]:
from langchain.document_loaders import TextLoader

In [7]:
loader = TextLoader("/Users/Hp/OneDrive/Desktop/LLM_OPS/data/Agentic AI.txt", encoding="utf8")
documents = loader.load()

In [14]:
documents[0].page_content[:87]  # Print the first 500 characters of the first documen

'Understanding Agentic Al\nAgentic AI refers to a new paradigm in artificial intelligence'

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

text_chunks=text_splitter.split_documents(documents)
len(text_chunks)

20

In [18]:
! uv pip install faiss-cpu

Using Python 3.13.2 environment at: C:\Users\Hp\OneDrive\Desktop\LLM_OPS\.venv
Resolved 3 packages in 354ms
Prepared 1 package in 3.54s
Installed 1 package in 309ms
 + faiss-cpu==1.12.0


In [23]:
! uv pip install langchain_google_genai

Using Python 3.13.2 environment at: C:\Users\Hp\OneDrive\Desktop\LLM_OPS\.venv
Resolved 39 packages in 5.99s
Prepared 13 packages in 10.11s
Installed 13 packages in 3.55s
 + cachetools==6.2.0
 + filetype==1.2.0
 + google-ai-generativelanguage==0.7.0
 + google-api-core==2.25.1
 + google-auth==2.41.1
 + googleapis-common-protos==1.70.0
 + grpcio==1.75.1
 + grpcio-status==1.75.1
 + langchain-google-genai==2.1.12
 + proto-plus==1.26.1
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.2
 + rsa==4.9.1


In [24]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

In [25]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [27]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)
vectorstore

In [28]:
retriever=vectorstore.as_retriever()

In [30]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Key Characteristics of Agentic AI Agentic Al systems are distinct from traditional Al models due to several core characteristics:
--------------------------------------------------
Document 2:
Agentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific tasks, but to operate autonomously towards
--------------------------------------------------
Document 3:
* Adaptability and Learning: They can adjust their behavior and improve their performance based on new information and experiences.
How Agentic AI Works
--------------------------------------------------
Document 4:
Understanding Agentic Al
--------------------------------------------------


In [31]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [32]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [33]:
from langchain.schema.output_parser import StrOutputParser
output_parser=StrOutputParser()

In [34]:

from langchain_google_genai import GoogleGenerativeAI

llm_model=GoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.0)

In [35]:
from langchain.schema.runnable import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [36]:

rag_chain.invoke("tell me about Agentic AI")

'Agentic AI is a new paradigm in artificial intelligence. These systems are designed to operate autonomously, going beyond simply responding to queries or performing specific tasks. They are distinct from traditional AI models due to several core characteristics. One key feature is their adaptability and capacity for learning. This means they can adjust their behavior and improve their performance based on new information and experiences.'